In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [25]:
df = pd.read_csv("kobe.csv")
data = df.dropna() # Suppression des lignes(les tirs) dont nous ne connaissons pas l'issue(marqué ou manqué)
qualitative_vars = ["action_type", "combined_shot_type", "game_event_id", "game_id", 
                    "period", "playoffs", "season", "shot_type",
                    "shot_zone_area", "shot_zone_basic", "shot_zone_range", "team_name", "game_date", "matchup", "opponent"]
quantitative_vars = [col for col in df.columns if col not in qualitative_vars + ["shot_made_flag", "shot_id", 'team_id']]  # garder le shot_id ? 

In [26]:
correlation_matrix = data[quantitative_vars+["shot_made_flag"]].corr()
correlation_matrix['shot_made_flag']

lat                  0.148070
loc_x               -0.000848
loc_y               -0.148070
lon                 -0.000848
minutes_remaining    0.028342
seconds_remaining    0.030804
shot_distance       -0.198242
shot_made_flag       1.000000
Name: shot_made_flag, dtype: float64

On observe que les variables qui jouent le plus sont 'lat' 'loc_y' et 'shot distance' On les inscrit dans qualitative_vars

In [27]:
quantitative_vars = ["lat", "loc_y", "shot_distance"]

In [28]:
X = data[quantitative_vars]  # Features : éléments ayant un impact lors de la prise du tir
y = data["shot_made_flag"]  # Target : issue du tir 

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.8, random_state=0)

# Réinitialiser les index pour éviter les problèmes de jointure
#X_train.reset_index(drop=True, inplace=True)

In [30]:
# Utilisation de OneHotEncoder pour les variables catégorielles
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), qualitative_vars)
    ],
    remainder='passthrough'
)

In [31]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

In [32]:
def apprentissage(preprocessor, classifier):
    # Création du pipeline
    pipeline = Pipeline([
        ('classifier', classifier)
    ])
    # Entraînement du modèle
    pipeline.fit(X_train, y_train)

    # Prédiction sur l'ensemble de test
    prediction = pipeline.predict(X_test)

    # Évaluation du modèle
    accuracy = accuracy_score(y_test, prediction)
    precision = precision_score(y_test, prediction, zero_division=0)
    recall = recall_score(y_test, prediction)    
    print(f"a = {accuracy}, p = {precision}, r = {recall}")

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

## Choix et optimisation de l'algorithme d'apprentissage

In [39]:
# Choix de l'algorithme de classification et de ses paramètres
for k in range(41, 91, 10):
    #classifier = LogisticRegression(max_iter=10000)
    classifier = KNeighborsClassifier(n_neighbors=k, p=1)
    #classifier = RandomForestClassifier(max_depth=6, random_state=0)
    print(f"========= Accuracy of our model for i = {k} ============")
    apprentissage(preprocessor, classifier)

========= Accuracy of our model for i = 41 ============
a = 0.5979180854168693, p = 0.5700063211125158, r = 0.3941215034965035
========= Accuracy of our model for i = 51 ============
a = 0.598307228329604, p = 0.5728725138571894, r = 0.3839597902097902
========= Accuracy of our model for i = 61 ============
a = 0.6005448000778286, p = 0.5812586445366529, r = 0.3673513986013986
========= Accuracy of our model for i = 71 ============
a = 0.6014203716314817, p = 0.5854136947218259, r = 0.35871940559440557
========= Accuracy of our model for i = 81 ============
a = 0.6018581574083082, p = 0.5881816523800839, r = 0.352381993006993


In [9]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# Supposons que X_train et y_train contiennent vos données d'entraînement

# Créer un objet de régression Lasso
lasso = Lasso(alpha=0.1)  # Vous pouvez ajuster alpha selon vos besoins

# Sélectionner les fonctionnalités les plus importantes
selector = SelectFromModel(lasso)
selector.fit(X_train, y_train)

# Obtenir les indices des fonctionnalités sélectionnées
selected_features_indices = selector.get_support(indices=True)

# Obtenir les noms des fonctionnalités sélectionnées
selected_features_names = X_train.columns[selected_features_indices]

# Créer un nouveau DataFrame avec seulement les fonctionnalités sélectionnées
X_train_selected = X_train[selected_features_names]


ValueError: could not convert string to float: 'Jump Shot'